In [1]:
import pandas as pd 
import numpy as np 
import sys
import dask.bag as db

import json
import dask.bag as db
import os

sys.path.append("..")

from pathlib import Path

import json

In [2]:
ai_category_list=['q-fin']
records=db.read_text("../input/arxiv/arxiv-metadata-oai-snapshot.json").map(lambda x:json.loads(x))
ai_docs = (records.filter(lambda x:any(ele in x['categories'] for ele in ai_category_list)==True))
get_metadata = lambda x: {'id': x['id'],
                  'title': x['title'],
                  'category':x['categories'],
                  'abstract':x['abstract'],
                 'version':x['versions'][-1]['created'],
                         'doi':x["doi"],
                         'authors_parsed':x['authors_parsed']}

data=ai_docs.map(get_metadata).to_dataframe().compute()

In [3]:
data.shape

(15741, 7)